In [11]:
%pip install ../../

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Processing /home/yuyang/SpuCo
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for spuco: filename=spuco-0.0.1-py3-none-any.whl size=81064 sha256=246ec6ec1405192c7f23bf42ab1d28030f0850e8959b86c39071405a1d064445
  Stored in directory: /tmp/pip-ephem-wheel-cache-anrcr9jm/wheels/99/d4/10/c6136b4f67d7a1fd0d788e21f761b3004ee3bbbfae90fc0ca1
Successfully built spuco
  Attempting uninstall: spuco
    Found existing installation: spuco 0.0.1
    Uninstalling spuco-0.0.1:
      Successfully uninstalled spuco-0.0.1
Note: you may need to restart the kernel to use updated packages.


In [1]:
import torch 

device = torch.device("cuda:7")

In [2]:
from spuco.utils import set_seed

set_seed(0)

/home/yuyang/SpuCo/.conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from wilds import get_dataset
import torchvision.transforms as transforms

# Load the full dataset, and download it if necessary
dataset = get_dataset(dataset="waterbirds", download=True, root_dir="/data")

transform = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
        ])

# Get the training set
train_data = dataset.get_subset(
    "train",
    transform=transform
)

# Get the training set
test_data = dataset.get_subset(
    "test",
    transform=transform
)

In [4]:
from spuco.utils import WILDSDatasetWrapper

trainset = WILDSDatasetWrapper(dataset=train_data, metadata_spurious_label="background", verbose=True)
testset = WILDSDatasetWrapper(dataset=test_data, metadata_spurious_label="background", verbose=True)

Partitioning data indices into groups: 100%|██████████| 5794/5794 [00:00<00:00, 2215296.02it/s]


In [5]:
trainset.group_weights

{(1, 1): 0.22043795620437956,
 (1, 0): 0.01167883211678832,
 (0, 0): 0.7295099061522419,
 (0, 1): 0.0383733055265902}

In [6]:
from spuco.utils import GroupLabeledDataset

invariant_trainset = GroupLabeledDataset(trainset, trainset.group_partition)

In [7]:
from spuco.models import model_factory 

model = model_factory("resnet50", trainset[0][0].shape, trainset.num_classes).to(device)

/home/yuyang/SpuCo/.conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/yuyang/SpuCo/.conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [8]:
from torch.optim import SGD
from spuco.invariant_train import GroupDRO 

group_dro = GroupDRO(
    model=model,
    num_epochs=300,
    trainset=invariant_trainset,
    batch_size=128,
    optimizer=SGD(model.parameters(), lr=1e-5, weight_decay=1.0, momentum=0.9),
    device=device,
    verbose=True
)
group_dro.train()

Epoch 0: 100%|██████████| 38/38 [00:13<00:00,  2.83batch/s, accuracy=35.59322033898305%, loss=0.719]


tensor([0.2419, 0.2365, 0.2585, 0.2631], device='cuda:7')


Epoch 1: 100%|██████████| 38/38 [00:10<00:00,  3.49batch/s, accuracy=50.847457627118644%, loss=0.709]


tensor([0.2426, 0.2321, 0.2584, 0.2669], device='cuda:7')


Epoch 2: 100%|██████████| 38/38 [00:10<00:00,  3.50batch/s, accuracy=62.71186440677966%, loss=0.681]


...

tensor([0.2962, 0.2048, 0.1958, 0.3033], device='cuda:7')


Epoch 298: 100%|██████████| 38/38 [00:10<00:00,  3.53batch/s, accuracy=98.30508474576271%, loss=0.198]


tensor([0.2957, 0.2049, 0.1961, 0.3034], device='cuda:7')


Epoch 299: 100%|██████████| 38/38 [00:10<00:00,  3.47batch/s, accuracy=100.0%, loss=0.18]    

tensor([0.2948, 0.2050, 0.1962, 0.3040], device='cuda:7')


In [9]:
from spuco.evaluate import Evaluator

evaluator = Evaluator(
    testset=testset,
    group_partition=testset.group_partition,
    group_weights=trainset.group_weights,
    batch_size=64,
    model=model,
    device=device,
    verbose=True
)
evaluator.evaluate()

Evaluating group-wise accuracy:  25%|██▌       | 1/4 [00:02<00:08,  2.74s/it]

Group (0, 0) Test Accuracy: 98.22616407982261


Evaluating group-wise accuracy:  50%|█████     | 2/4 [00:05<00:05,  2.64s/it]

Group (0, 1) Test Accuracy: 89.490022172949


Evaluating group-wise accuracy:  75%|███████▌  | 3/4 [00:06<00:02,  2.05s/it]

Group (1, 0) Test Accuracy: 85.202492211838


Evaluating group-wise accuracy: 100%|██████████| 4/4 [00:07<00:00,  1.98s/it]

Group (1, 1) Test Accuracy: 92.05607476635514


{(0, 0): 98.22616407982261,
 (0, 1): 89.490022172949,
 (1, 0): 85.202492211838,
 (1, 1): 92.05607476635514}